In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# reading the excel file
df_start = pd.read_excel('worldcities.xlsx')

#get a data frame with selected columns
FORMAT = ['city', 'lat', 'lng','country','iso2','population']
df_selected = df_start[FORMAT]

#insert in the dataframe the flag for the population
df_selected["flg_pop"]=df_selected.population.apply(lambda x: 1 if x > 200000 else 0 )

# sorting values by longitude
df_selected.sort_values(by=['lng'], inplace=True, ignore_index=True)

# searching the index of London (GB)
rslt_df = df_selected.loc[(df_selected['city'] == 'London') & 
              (df_selected['iso2']=='GB')] 
index_of_london = rslt_df.index[0]

# splitting the dataframe before and after the London index
df_a = df_selected.iloc[:index_of_london]
df_b = df_selected.iloc[index_of_london:]

# reshaping of dataframe
df = df_b.append(df_a).reset_index(drop=True)

In [ ]:
class AroundTheWorld(object):
    
    def __init__(self,dataframe, city_start, country_start, n_min=3, x_size=0.2, y_size=0.1):
        self.dataframe=dataframe
        self.coord_step=None #(lng, lat city_step)
        self.lat_max=None
        self.lat_min=None
        self.lng_min=None
        self.lng_max=None
        self.city_start=city_start
        self.country_start=country_start
        self.city_step=city_start
        self.country_step=country_start
        self.n_min=n_min
        self.x_size=x_size
        self.y_size=y_size
        self.hours=0
        self.n_step=0
        
    def generate_grid(self):
        self.coord_step = tuple(self.dataframe.loc[(self.dataframe['city'] == self.city_step) & (self.dataframe['iso2']== self.country_step)][['lng','lat']].iloc[0]) #midpoint of shortest side of rectangular
        self.lat_max=self.coord_step[1]+self.y_size/2
        self.lat_min=self.coord_step[1]-self.y_size/2
        self.lng_min=self.coord_step[0]
        self.lng_max=self.coord_step[0]+self.x_size
        # print(self.lat_max, self.lat_min, self.lng_min, self.lng_max)
        
    def query(self): 
        grid_city=self.dataframe.loc[(self.dataframe['lat'] >= self.lat_min) &
                                     (self.dataframe['lat'] <= self.lat_max) & 
                                     (self.dataframe['lng'] <= self.lng_max) & 
                                     (self.dataframe['lng'] >= self.lng_min) & 
                                     (self.dataframe['city'] != self.city_step)]
        #print(grid_city)
        return grid_city
    
    def check_grid_city(self):
        self.generate_grid()
        grid_city=self.query()
        n_row=grid_city.shape[0]
        if n_row < self.n_min:
            self.x_size *= 2
            self.y_size *= 2
            return self.check_grid_city() 
        else: 
            return grid_city
        print(n_row)
        
    
    def weight(self, grid_city):
        grid_city["distance"]=grid_city.apply(self.calculate_distance, axis=1, point_city_step=self.coord_step )
        grid_city.sort_values(by=['distance'], inplace=True)
        grid_city=grid_city[:self.n_min]
        grid_city["weight"] = [2**x for x in range(1,self.n_min+1)]
        grid_city["weight"] += grid_city.apply(self.calculate_weigth, axis=1, country_step=self.country_step)
        return grid_city
        
    @staticmethod #decorator
    def calculate_weigth(row, country_step):
        pop_weigth=2 if row['flg_pop'] == 1 else 0
        country_weigth=2 if row['iso2'] != country_step else 0
        return pop_weigth + country_weigth
    
    @staticmethod
    def calculate_distance(row, point_city_step):
        return euclidean_distances([list(point_city_step)], [[row['lng'],row['lat']]])[0][0]
             
    def step(self, grid_city):
        grid_city.sort_values(by=["weight","distance"], ascending=[True, False], inplace=True)
        step=grid_city.iloc[0]
        self.coord_step=tuple(step[["lng", "lat"]])
        self.city_step=step["city"]
        self.country_step=step["iso2"]
        self.hours += grid_city["weight"]
        print(self.coord_step, self.city_step, self.country_step, self.hours)
        
    def travel(self):
        while ((self.city_start != self.city_step) or (self.country_start != self.country_step) or (self.n_step == 0)):
            grid_city = self.check_grid_city()
            grid_city_weight= self.weight(grid_city)
            self.step(grid_city_weight)
            self.n_step += 1
        print(self.hours, self.n_step, self.city_step, self.country_step)
            

In [ ]:
around = AroundTheWorld(df, "London", "GB")
#around.generate_grid()
#k=around.query()
#around.check_grid_city()
#h=around.weight(k)
#around.step(h)
around.travel()

In [ ]:
plt.figure(figsize=(200,100))
plt.scatter(df.lng,df.lat, color="green", marker="o", alpha=0.5)
plt.scatter(k.lng,k.lat,color="red", marker="+")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid()
plt.show()